In [ ]:
import numpy as np,os
import random
import nibabel as nb
import glob
import cv2 as cv
import glob
import matplotlib
from matplotlib import pyplot as plt
from PIL import Image

In [ ]:
num2label={'58':1,'86':2,'170':3,'30324':4,'30325':5,'187':6,'40357':7,'40358':8,'237':9,'29193':10,'1302':11,
           '1326':12,'29662':13,'29663':14,'480':15,'2473':16,'32248':17,'32249':18,'7578':19,'1247':20}
num_list=['58','86','170','30324','30325','187','40357','40358','237','29193',
          '1302','1326','29662','29663','480','2473','32248','32249','7578','1247']
volumes_path="/home/qinji/Visceral/SilverCorpus/Volumes/"
seg_path="/home/qinji/Visceral/SilverCorpus/Segmentations/"
total_volumes=glob.glob(os.path.join(volumes_path,'*_1_CTce_ThAb*'))


In [ ]:
print(len(total_volumes))
image_save_path='/home/qinji/Local_prototype_network/data/Visceral/Organ_images/'
seg_save_path="/home/qinji/Local_prototype_network/data/Visceral/SegmentationClass/"
os.makedirs(image_save_path)
os.makedirs(seg_save_path)

In [ ]:
def get_range(volume):
    _,_,batch = volume.shape
    slice_with_class = np.sum(volume.reshape(-1,batch), axis=0) > 150
    range_index=[]
    for i in range(batch-1):
        if slice_with_class[i]==True:
            range_index.append(i)
    if len(range_index)>0:
        print(min(range_index),max(range_index))
    if len(range_index)>20:
        range_index=random.sample(range_index,5)
    return range_index

def save_img(img,save_path):
    try:
        cv.imwrite(save_path,img)
    except:
        print('Image is empty')

def gray_scale(img_gray):
    #img_gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    if len(img_gray.shape)==2:
        rows,cols = img_gray.shape
    else:
        print('Wrong!')
        return img_gray
    im1 = img_gray.astype(float)
    A,B=im1.min(),im1.max()
    im1 -= A
    im1 /= B-A
    im1 *= 255
    return im1

In [ ]:
for volume_path in total_volumes:
    volume_nifty = nb.load(volume_path)
    volume = volume_nifty.get_fdata()
    print(volume.shape)
    pid=volume_path.split('/')[-1].split('_')[0]
    segmentations=glob.glob(os.path.join(seg_path,pid+'_*'))
    for segmentation_path in segmentations:
        num=segmentation_path.split('/')[-1].split('_')[2]
        if num in num_list:
            label=num2label[num]
        else:
            continue
        print(pid,num,label)
        seg_nifty=nb.load(segmentation_path)
        seg_label=seg_nifty.get_fdata()
        range_index=get_range(seg_label)
        if len(range_index)==0:
            continue
        
        #print(segmentation_path)
        count=0
        for index in range_index:
            seg_save=seg_label[:,:,index].transpose()
            seg_save=label*(seg_save>0.1)
            volume_name='_'.join([pid,str(label),str(count)])+'.jpg'
            seg_name='_'.join([pid,str(label),str(count)])+'.png'
            save_img(gray_scale(volume[:,:,index].transpose()),os.path.join(image_save_path,volume_name))
            save_img(seg_save,os.path.join(seg_save_path,seg_name))
            count+=1

In [ ]:
txt_path="/home/qinji/Local_prototype_network/data/Visceral/Segmentation"
os.makedirs(txt_path)

In [ ]:
train_images=os.listdir(image_save_path)
txt_name=os.path.join(txt_path,'trainaug.txt')
with open(txt_name,'a') as file_handle:
    for image in train_images:
        class_label=image.split('_')[1]
        if class_label=='checkpoints':
            continue
        file_handle.writelines(image.split('.')[0])     
        file_handle.writelines('\n')

In [ ]:
train_images=os.listdir(image_save_path)
image_list=[[] for i in range(20)]
for image in train_images:
    class_label=image.split('_')[1]
    if class_label=='checkpoints':
        continue
    image_list[int(class_label)-1].append(image)
sum=0
for i in range(20):
    sum+=len(image_list[i])
print(sum)

In [ ]:
os.makedirs(os.path.join(txt_path,'trainaug'))
image_list=[[] for i in range(20)]
for image in train_images:
    class_label=image.split('_')[1]
    if class_label=='checkpoints':
        continue
    image_list[int(class_label)-1].append(image)

for num in num_list:
    i=num2label[num]
    txt_name=os.path.join(txt_path,'trainaug','class'+str(i)+'.txt')
    with open(txt_name,'a') as f:
        for j in range(len(image_list[i-1])):
            f.writelines(image_list[i-1][j].split('.')[0])
            f.writelines('\n')